### Key interactions to json

This notebook converts the key interactions information obtained in csv to json format for subsequent comparison.

In [1]:
import pandas as pd

df = pd.read_csv('./keyinteractions.csv')
df

,Target,H-Bond,H-Bond acceptor,H-Bond donor,Halogen bond,Hydrophobic contact,Pi-cation lAro,Pi-cation pAro,Pi-stacking,Salt Bridge lneg,Salt Bridge pneg
0,5HT2A,NaN,NaN,NaN,NaN,Trp336/Phe339,NaN,NaN,NaN,NaN,Asp155
1,5HT2A-AF,NaN,NaN,NaN,NaN,Trp336/Phe339,NaN,NaN,NaN,NaN,Asp155
2,BCL2,NaN,NaN,NaN,NaN,Phe104;Tyr108+Arg146;Phe153;Asp111+Phe112,NaN,NaN,NaN,NaN,NaN
3,BRAF,Cys532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BRD4,Asn140,NaN,NaN,NaN,Trp81/Pro82/Phe83;Leu92/Leu94/Val87/Tyr97,NaN,NaN,NaN,NaN,NaN
5,BRD4-holo,Asn140,NaN,NaN,NaN,Trp81/Pro82/Phe83;Leu92/Leu94/Val87/Tyr97,NaN,NaN,NaN,NaN,NaN
6,BTK,Met477,Met477,Met477,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Beta2AR,Ser203+Ser207;Asn379+Tyr383,NaN,NaN,NaN,NaN,NaN,NaN,Phe357,NaN,NaN
8,DRD2,NaN,NaN,NaN,NaN,Trp386,NaN,NaN,NaN,NaN,Asp114
9,HDAC6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PHE583/Phe643,NaN,NaN


In [2]:
def parse_rule(rule, target):

    def add_prefix(r, target):
        upper = r.upper()
        pref = 'A_' if target != 'MEK1' else 'B_' # MEK1 in chain B
        if "'" in upper: # NAMPT in PPI surface
            upper = upper.replace("'", "")
            pref = 'B_'
        return pref + upper

    rules = []
    for sub_rule in rule.split(';'):
        if '+' in sub_rule:
            res = sub_rule.split('+')
            match_mode = 'all'
        elif '/' in sub_rule:
            res = sub_rule.split('/')
            match_mode = 'any'
        else:
            res = [sub_rule]
            match_mode = 'all'
        rules.append({
            'residues': [add_prefix(r, target) for r in res],
            'match_mode': match_mode
        })
    return rules

In [3]:
inters = {}
for index, row in df.iterrows():
    target = row['Target']
    inters[target] = {
        col: parse_rule(value, target) 
        for col, value in row[1:].items() if not pd.isna(value)}

In [4]:
import json
with open('key_interactions.json', 'w') as f:
    json.dump(inters, f, indent=2, sort_keys=True)